# Imports

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt

from pandas.plotting import register_matplotlib_converters
from IPython.display import display
# from tsa_functions import *
from tsa_tools import *  # See last cell

register_matplotlib_converters()
sns.set_style('darkgrid')

np.set_printoptions(precision=4)
pd.set_option('precision', 4)

import statsmodels.api as sm
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.exponential_smoothing.ets import ETSModel

# Loading the Data

In [68]:
df_calendar = pd.read_csv('data/m5/calendar.csv')
df_price = pd.read_csv('data/m5/sell_prices.csv')
df_sales = pd.read_csv('data/m5/sales_train_validation.csv')

In [66]:
full_df = (df_sales.set_index([*df_sales.columns[5::-1]]).T
           .set_index(pd.DatetimeIndex(df_calendar.date)))

ValueError: Length mismatch: Expected 1941 rows, received array of length 1969

In [67]:
df_calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [58]:
lvl9 = full_df.sum(level=[0, 1, 2, 3], axis=1)

state_id          CA                                                    \
store_id        CA_1                                                     
cat_id       HOBBIES             HOUSEHOLD               FOODS           
dept_id    HOBBIES_1 HOBBIES_2 HOUSEHOLD_1 HOUSEHOLD_2 FOODS_1 FOODS_2   
date                                                                     
2011-01-29       528        28         361         181     297     674   
2011-01-30       489         9         350         170     284     655   
2011-01-31       409         6         279         114     214     396   
2011-02-01       383         9         278         123     175     476   
2011-02-02       263         5         195         135     182     354   
...              ...       ...         ...         ...     ...     ...   
2016-04-20       333        30         665         204     287     423   
2016-04-21       380        30         580         186     297     365   
2016-04-22       471        56         695         205     331     489   
2016-04-23       621        49         933         319     419     590   
2016-04-24       641        73        1022         324     329     734   

state_id                                            ...      WI          \
store_id                CA_2                        ...    WI_2           
cat_id               HOBBIES             HOUSEHOLD  ...   FOODS           
dept_id    FOODS_3 HOBBIES_1 HOBBIES_2 HOUSEHOLD_1  ... FOODS_1 FOODS_2   
date                                                ...                   
2011-01-29    2268       522        16         529  ...     191     155   
2011-01-30    2198       381        16         461  ...     184     128   
2011-01-31    1398       352        16         306  ...     154     199   
2011-02-01    1607       344         6         270  ...     205     244   
2011-02-02    1496       283        13         217  ...      96     147   
...            ...       ...       ...         ...  ...     ...     ...   
2016-04-20    1780       303        41         586  ...     335     830   
2016-04-21    1871       232        39         521  ...     307     851   
2016-04-22    2140       425        60         778  ...     400     953   
2016-04-23    2646       505        37        1096  ...     408     939   
2016-04-24    2990       447        60        1131  ...     347     932   

state_id                                                                \
store_id                WI_3                                             
cat_id               HOBBIES             HOUSEHOLD               FOODS   
dept_id    FOODS_3 HOBBIES_1 HOBBIES_2 HOUSEHOLD_1 HOUSEHOLD_2 FOODS_1   
date                                                                     
2011-01-29    1269       256        22         584         148     152   
2011-01-30    1121       342        14         541         195     138   
2011-01-31    1233       228        20         420         106     127   
2011-02-01    1564       183        11         327          94      98   
2011-02-02     724        70         4         151          53      87   
...            ...       ...       ...         ...         ...     ...   
2016-04-20    1840       202        42         535         148     180   
2016-04-21    1920       182        21         573         142     227   
2016-04-22    2066       264        32         701         166     261   
2016-04-23    2404       381        32         801         232     324   
2016-04-24    2380       230        21         766         201     317   

state_id                    
store_id                    
cat_id                      
dept_id    FOODS_2 FOODS_3  
date                        
2011-01-29     583    2293  
2011-01-30     585    2383  
2011-01-31     575    1841  
2011-02-01     533    1965  
2011-02-02     340    1427  
...            ...     ...  
2016-04-20     473    1579  
2016-04-21     411    1670  
2016-04-22     507    1897  
2016-04-23     505    2

# Pre-procesing